In [1]:
!uv pip install datasets

Using Python 3.10.18 environment at: /mnt/c/Users/LANDSOFT/Documents/dev/law/.venv
Audited 1 package in 238ms


In [2]:
from pathlib import Path
import zipfile
import json
from datasets import Dataset
from tqdm.auto import tqdm

/mnt/c/Users/LANDSOFT/Documents/dev/law/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BASE_DIR = "/mnt/d/data/06.계약 외 법률 문서 서식 데이터"

In [4]:
# BASE_DIR 아래 모든 zip file 경로 수집
zip_files = list(Path(BASE_DIR).rglob("*.zip"))
print(f"Found {len(zip_files)} zip files.")
for zip_file in zip_files:
    print(zip_file)

Found 44 zip files.
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_내용증명_기타_내용증명.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_민원행정_동의_위임관련문서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_불복신청_불복이유서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_불복신청_항소_상고_항고장.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_소의개시_종료_소의개시.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_소의개시_종료_소의종료.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_민원행정_요청_신고관련문서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_내용증명_기타_기타.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_소의진행_법적동의_위임관련문서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_소의진행_신청_청구관련문서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_소의진행_진술_의견관련문서.zip
/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.

In [5]:
# ZIP 파일 내용 확인
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"ZIP 파일 내부 파일 목록 ({zip_file}):")
        for file in file_list[:10]:  # 처음 10개만 출력
            print(file)

ZIP 파일 내부 파일 목록 (/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_내용증명_기타_내용증명.zip):
/내용증명_0005.json
/내용증명_0003.json
/내용증명_0010.json
/내용증명_0011.json
/내용증명_0006.json
/내용증명_0008.json
/내용증명_0002.json
/내용증명_0018.json
/내용증명_0007.json
/내용증명_0016.json
ZIP 파일 내부 파일 목록 (/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_민원행정_동의_위임관련문서.zip):
/동의서_0005.json
/동의서_0002.json
/위임장_0002.json
/동의서_0007.json
/동의서_0004.json
/위임장_0003.json
/증명서_원_0001.json
/위임장_0008.json
/동의서_0008.json
/동의서_0009.json
ZIP 파일 내부 파일 목록 (/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_불복신청_불복이유서.zip):
/항소_항고_불기소이유서_0037.json
/항소_항고_불기소이유서_0013.json
/항소_항고_불기소이유서_0030.json
/항소_항고_불기소이유서_0014.json
/항소_항고_불기소이유서_0011.json
/항소_항고_불기소이유서_0010.json
/항소_항고_불기소이유서_0046.json
/항소_항고_불기소이유서_0008.json
/항소_항고_불기소이유서_0021.json
/항소_항고_불기소이유서_0029.json
ZIP 파일 내부 파일 목록 (/mnt/d/data/06.계약 외 법률 문서 서식 데이터/3.개방데이터/1.데이터/Training/01.원천데이터/TS_불복신청_항소_상고_항고장.zip):
/항고장_0013.json
/항고장_0022.json


In [6]:
# TL JSON 구조 확인
for zip_file in zip_files:
    if "TL" in zip_file.name or "VL" in zip_file.name:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            json_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
            print(f"\n{zip_file.name}의 JSON 파일: {json_files}")
            
            if json_files:
                with zip_ref.open(json_files[0]) as f:
                    data = json.load(f)
                    print(f"JSON 타입: {type(data)}")
                    if isinstance(data, dict):
                        print(f"Keys: {list(data.keys())[:10]}")
                    elif isinstance(data, list):
                        print(f"리스트 길이: {len(data)}")
                        print(f"첫 번째 항목 샘플: {str(data[0])[:200]}")


TL_내용증명_기타_기타.zip의 JSON 파일: ['/기타_0005.json', '/기타_0009.json', '/기타_0010.json', '/기타_0004.json', '/기타_0012.json', '/기타_0007.json', '/기타_0008.json', '/기타_0001.json', '/기타_0002.json']
JSON 타입: <class 'dict'>
Keys: ['document']

TL_내용증명_기타_내용증명.zip의 JSON 파일: ['/내용증명_0008.json', '/내용증명_0023.json', '/내용증명_0003.json', '/내용증명_0013.json', '/내용증명_0015.json', '/내용증명_0002.json', '/내용증명_0045.json', '/내용증명_0050.json', '/내용증명_0006.json', '/내용증명_0034.json', '/내용증명_0021.json', '/내용증명_0042.json', '/내용증명_0012.json', '/내용증명_0011.json', '/내용증명_0004.json', '/내용증명_0032.json', '/내용증명_0040.json', '/내용증명_0018.json', '/내용증명_0020.json', '/내용증명_0041.json', '/내용증명_0031.json', '/내용증명_0005.json', '/내용증명_0022.json', '/내용증명_0033.json', '/내용증명_0017.json', '/내용증명_0029.json', '/내용증명_0072.json', '/내용증명_0051.json', '/내용증명_0007.json', '/내용증명_0010.json', '/내용증명_0026.json', '/내용증명_0030.json', '/내용증명_0064.json', '/내용증명_0082.json', '/내용증명_0027.json', '/내용증명_0097.json', '/내용증명_0016.json', '/내용증명_0062.json', '/내용증명_0048.json', '

In [7]:
# JSON 데이터 구조 상세 확인
for zip_file in zip_files:
    if "TL" in zip_file.name or "VL" in zip_file.name:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            json_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
            
            for json_file in json_files:
                with zip_ref.open(json_file) as f:
                    data = json.load(f)
                    print(f"\n{zip_file.name} - {json_file} 데이터 구조:")
                    
                    if isinstance(data, list):
                        for i, item in enumerate(data[:2]):  # 처음 2개 항목만 확인
                            print(f"\n항목 {i} 타입: {type(item)}")
                            if isinstance(item, dict):
                                print(f"Keys: {list(item.keys())[:10]}")
                                for key in list(item.keys())[:5]:  # 처음 5개 키의 값 샘플 출력
                                    print(f"{key}: {str(item[key])[:200]}")
                    elif isinstance(data, dict):
                        print(f"Keys: {list(data.keys())[:10]}")
                        for key in list(data.keys())[:5]:  # 처음 5개 키의 값 샘플 출력
                            print(f"{key}: {str(data[key])[:200]}")


TL_내용증명_기타_기타.zip - /기타_0005.json 데이터 구조:
Keys: ['document']
document: {'type': '계약 외 법률 문서', 'metadata': {'file_info': {'document_name': '계약외기타_0005', 'document_size': '166kb', 'document_page_count': '4', 'document_total_ids': '31', 'document_format': 'pdf', 'document_c

TL_내용증명_기타_기타.zip - /기타_0009.json 데이터 구조:
Keys: ['document']
document: {'type': '계약 외 법률 문서', 'metadata': {'file_info': {'document_name': '계약외기타_0009', 'document_size': '305kb', 'document_page_count': '9', 'document_total_ids': '44', 'document_format': 'pdf', 'document_c

TL_내용증명_기타_기타.zip - /기타_0010.json 데이터 구조:
Keys: ['document']
document: {'type': '계약 외 법률 문서', 'metadata': {'file_info': {'document_name': '계약외기타_0010', 'document_size': '84kb', 'document_page_count': '2', 'document_total_ids': '21', 'document_format': 'pdf', 'document_cr

TL_내용증명_기타_기타.zip - /기타_0004.json 데이터 구조:
Keys: ['document']
document: {'type': '계약 외 법률 문서', 'metadata': {'file_info': {'document_name': '계약외기타_0004', 'document_size': '61kb', 'do

In [8]:
from glob import glob
from pathlib import Path
import json, zipfile
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict

# 2) split 매핑
split_map = {"TL":"train_long", "TS":"train_short", "VL":"valid_long", "VS":"valid_short"}

# 3) 누적 버퍼
rows_by_split = {v: [] for v in split_map.values()}

def normalize_item(item, src_zip, src_json, is_short=False):
    """
    다양한 JSON 구조를 단일 row로 정규화.
    """
    row = {"source_zip": src_zip, "source_json": src_json}

    if isinstance(item, dict):
        # raw_json 저장
        row["raw_json"] = json.dumps(item, ensure_ascii=False)
        
        # document 구조 처리
        if "document" in item and isinstance(item["document"], dict):
            doc = item["document"]
            
            # short 스플릿
            if is_short:
                row["doc_type"] = doc.get("doc_type")
                row["title"] = doc.get("title")
                row["page_count"] = doc.get("page_count")
                row["sub_document_count"] = doc.get("sub_document_count")
                
                # sub_documents에서 텍스트 추출
                sub_docs = doc.get("sub_documents", [])
                if sub_docs:
                    text_parts = []
                    for sub in sub_docs:
                        if isinstance(sub, dict) and "contents" in sub:
                            for content in sub["contents"]:
                                if isinstance(content, dict) and "text" in content:
                                    text_parts.append(content["text"])
                    row["text"] = "\n".join(text_parts) if text_parts else None
                else:
                    row["text"] = None
                
                return row
            
            # long 스플릿
            else:
                meta = doc.get("metadata", {})
                file_info = meta.get("file_info", {}) if isinstance(meta, dict) else {}
                row["document_name"] = file_info.get("document_name")
                
                # 카테고리 정보
                doc_category = file_info.get("document_category", {}) if isinstance(file_info, dict) else {}
                row["main_category"] = doc_category.get("main_category") if isinstance(doc_category, dict) else None
                row["sub_category"] = doc_category.get("sub_category") if isinstance(doc_category, dict) else None
                row["detail_category"] = doc_category.get("detail_category") if isinstance(doc_category, dict) else None

                # 텍스트 추출 시도
                text = doc.get("text") or doc.get("content") or doc.get("body")
                
                # sections 구조
                if not text:
                    sections = doc.get("sections", [])
                    if sections:
                        text_parts = []
                        for section in sections:
                            if isinstance(section, dict):
                                content = section.get("content", {})
                                if isinstance(content, dict) and "description" in content:
                                    text_parts.append(content["description"])
                        text = "\n".join(text_parts) if text_parts else None
                
                row["text"] = text
                return row
        
        # 직접 text 필드가 있는 경우
        if "text" in item and isinstance(item["text"], str):
            row["text"] = item["text"]
            return row
        
        # 텍스트 없음
        row["text"] = None
        return row

    # list/str/기타
    row["raw_json"] = json.dumps(item, ensure_ascii=False) if not isinstance(item, str) else item
    row["text"] = None
    return row

# 4) ZIP 열고 JSON 적재
for zpath in tqdm(zip_files, desc="loading zips"):
    key = next((k for k in split_map if k in zpath.name), None)
    if not key:
        continue
    split = split_map[key]
    is_short = "short" in split

    with zipfile.ZipFile(zpath, "r") as zf:
        json_names = [n for n in zf.namelist() if n.endswith(".json")]
        for jn in json_names:
            with zf.open(jn) as f:
                data = json.load(f)

            if isinstance(data, list):
                for item in data:
                    rows_by_split[split].append(
                        normalize_item(item, zpath.name, jn, is_short=is_short)
                    )
            else:
                rows_by_split[split].append(
                    normalize_item(data, zpath.name, jn, is_short=is_short)
                )

# 5) DatasetDict 생성
dataset_dict = DatasetDict({
    sp: Dataset.from_list(rows) for sp, rows in rows_by_split.items() if len(rows) > 0
})

# 6) 컬럼 추가 - long
def _add_cols_long(example):
    effective_text = example.get("text")
    char_len = len(effective_text) if isinstance(effective_text, str) else 0
    word_len = len(effective_text.split()) if isinstance(effective_text, str) else 0

    return {
        "text": effective_text,
        "char_len": char_len,
        "word_len": word_len,
        "document_name": example.get("document_name"),
        "main_category": example.get("main_category"),
        "sub_category": example.get("sub_category"),
        "detail_category": example.get("detail_category"),
    }

# 7) 컬럼 추가 - short
def _add_cols_short(example):
    effective_text = example.get("text")
    char_len = len(effective_text) if isinstance(effective_text, str) else 0
    word_len = len(effective_text.split()) if isinstance(effective_text, str) else 0

    return {
        "text": effective_text,
        "char_len": char_len,
        "word_len": word_len,
        "doc_type": example.get("doc_type"),
        "title": example.get("title"),
        "page_count": example.get("page_count"),
        "sub_document_count": example.get("sub_document_count"),
    }

# 8) split별로 적용
for sp, ds in list(dataset_dict.items()):
    if "short" in sp:
        ds = ds.map(_add_cols_short, desc=f"add columns [{sp}]")
    else:
        ds = ds.map(_add_cols_long, desc=f"add columns [{sp}]")
    
    ds = ds.add_column("split", [sp] * ds.num_rows)
    ds = ds.add_column("row_id", [f"{sp}-{i}" for i in range(ds.num_rows)])
    dataset_dict[sp] = ds

print(dataset_dict)

add columns [valid_short]: 100%|██████████| 1025/1025 [00:00<00:00, 14763.50 examples/s]

DatasetDict({
    train_long: Dataset({
        features: ['source_zip', 'source_json', 'raw_json', 'document_name', 'main_category', 'sub_category', 'detail_category', 'text', 'char_len', 'word_len', 'split', 'row_id'],
        num_rows: 8233
    })
    train_short: Dataset({
        features: ['source_zip', 'source_json', 'raw_json', 'doc_type', 'title', 'page_count', 'sub_document_count', 'text', 'char_len', 'word_len', 'split', 'row_id'],
        num_rows: 8233
    })
    valid_long: Dataset({
        features: ['source_zip', 'source_json', 'raw_json', 'document_name', 'main_category', 'sub_category', 'detail_category', 'text', 'char_len', 'word_len', 'split', 'row_id'],
        num_rows: 1025
    })
    valid_short: Dataset({
        features: ['source_zip', 'source_json', 'raw_json', 'doc_type', 'title', 'page_count', 'sub_document_count', 'text', 'char_len', 'word_len', 'split', 'row_id'],
        num_rows: 1025
    })
})


In [9]:
dataset_dict['train_long'][0]

{'source_zip': 'TL_내용증명_기타_기타.zip',
 'source_json': '/기타_0005.json',
 'raw_json': '{"document": {"type": "계약 외 법률 문서", "metadata": {"file_info": {"document_name": "계약외기타_0005", "document_size": "166kb", "document_page_count": "4", "document_total_ids": "31", "document_format": "pdf", "document_creation_date": "2024-11-14", "document_category": {"main_category": "내용증명_기타", "sub_category": "기타", "detail_category": "기타"}, "additional_info": {"related_law": "헌법재판소법", "document_purpose": "헌법소원을 통해 특정 법률 조항이 헌법에 위반되는지 여부를 판단받아 해당 조항의 효력을 다투기 위함", "document_use_case": "법률이 특정 권리를 침해한다고 판단될 때, 해당 법률 조항에 대한 헌법소원을 통해 헌법재판소의 판단을 받기 위해 활용.", "public_domain_sources": null}}}, "sections": [{"id": "1", "format": {"position": {"line": "1"}, "section_type": "TEXT", "format_type": "제목", "is_article_title": false, "article": null, "paragraph": null, "subparagraph": null}, "content": {"content_labels": ["기타"], "description": "헌법소원 심판청구\\n"}}, {"id": "2", "format": {"position": {"line": "2"}, "section_type

In [10]:
dataset_dict['train_short'][0]

{'source_zip': 'TS_내용증명_기타_내용증명.zip',
 'source_json': '/내용증명_0005.json',
 'raw_json': '{"document": {"doc_type": "2036", "title": null, "status": 10, "is_hold": null, "page_count": 3, "sub_document_count": 15, "created_time": "2024-10-17T15:25:54.866", "main_category": null, "sub_category": null, "detail_category": null, "related_law": null, "purpose": null, "use_case": null, "sub_documents": [{"id": 1, "type": "TEXT", "format_content": "2036AAA", "depth": 0, "sort_order": 2, "article": null, "paragraph": null, "subParagraph": null, "is_article_title": false, "content_labels": ["2036001"], "note": "", "name": null, "contents": [{"text": "수  신 :    주식회사 (@회사명1)\\n사업자등록번호: (@고유번호1)\\n(@주소1)\\n대표: (@성명1)", "type": null, "rowId": null, "cells": null}], "coordinate": null}, {"id": 5, "type": "TEXT", "format_content": "2036AAA", "depth": 0, "sort_order": 3, "article": null, "paragraph": null, "subParagraph": null, "is_article_title": false, "content_labels": ["2036002"], "note": "", "name": 

In [11]:
dataset_dict['valid_long'][0]

{'source_zip': 'VL_내용증명_기타_기타.zip',
 'source_json': '/기타_0003.json',
 'raw_json': '{"document": {"type": "계약 외 법률 문서", "metadata": {"file_info": {"document_name": "계약외기타_0003", "document_size": "73kb", "document_page_count": "2", "document_total_ids": "25", "document_format": "pdf", "document_creation_date": "2024-11-14", "document_category": {"main_category": "내용증명_기타", "sub_category": "기타", "detail_category": "기타"}, "additional_info": {"related_law": "헌법재판소법, 형사소송법", "document_purpose": "피고인이 특정 법률 조항이 헌법에 위반된다고 주장하며 해당 조항의 위헌 여부를 헌법재판소에 심판을 제청하도록 법원에 신청하기 위함", "document_use_case": "형사 사건에서 피고인이 특정 법률 조항의 위헌 여부를 다투기 위해, 해당 조항이 헌법에 위반된다고 주장하며 헌법재판소에 심판을 제청할 때 활용. 이는 특히 형사소송에서 피고인의 방어권 강화와 법률의 헌법적 타당성을 확인하는 데 사용됨.", "public_domain_sources": null}}}, "sections": [{"id": "1", "format": {"position": {"line": "1"}, "section_type": "TEXT", "format_type": "제목", "is_article_title": false, "article": null, "paragraph": null, "subparagraph": null}, "content": {"content_labels": ["기타"], "descrip

In [12]:
dataset_dict['valid_short'][0]

{'source_zip': 'VS_내용증명_기타_기타.zip',
 'source_json': '/기타_0003.json',
 'raw_json': '{"document": {"doc_type": "2037", "title": "위헌법률 심판제청 신청서", "status": 4, "is_hold": null, "page_count": 2, "sub_document_count": 25, "created_time": "2024-11-14T17:34:02.968", "main_category": null, "sub_category": null, "detail_category": null, "related_law": null, "purpose": null, "use_case": null, "sub_documents": [{"id": 1, "type": "TEXT", "format_content": "2037AAA", "depth": 0, "sort_order": 1, "article": null, "paragraph": null, "subParagraph": null, "is_article_title": false, "content_labels": ["2037036"], "note": "", "name": null, "contents": [{"text": "위헌법률 심판제청 신청서", "type": null, "rowId": null, "cells": null}], "coordinate": null}, {"id": 2, "type": "TEXT", "format_content": "2037AAB", "depth": 0, "sort_order": 2, "article": null, "paragraph": null, "subParagraph": null, "is_article_title": false, "content_labels": ["2037005", "2037006"], "note": "", "name": null, "contents": [{"text": "사    

In [13]:
dataset_dict.save_to_disk(BASE_DIR + "/processed_datasets")

Saving the dataset (1/1 shards): 100%|██████████| 1025/1025 [00:00<00:00, 8176.64 examples/s]


In [15]:
# # Remove the problematic column before pushing
# dataset_dict['train_short'] = dataset_dict['train_short'].remove_columns(['metadata'])
# dataset_dict['valid_short'] = dataset_dict['valid_short'].remove_columns(['metadata'])
# dataset_dict['train_long'] = dataset_dict['train_long'].remove_columns(['metadata'])
# dataset_dict['valid_long'] = dataset_dict['valid_long'].remove_columns(['metadata'])

In [17]:
import os

# 각 데이터셋을 별도 config로 업로드
print("train_short 업로드 중...")
dataset_dict['train_short'].push_to_hub(
    "brainer/non_contractual_legal_data", 
    config_name="short",
    split="train",
    num_proc=os.cpu_count()
)

print("valid_short 업로드 중...")
dataset_dict['valid_short'].push_to_hub(
    "brainer/non_contractual_legal_data", 
    config_name="short",
    split="validation",
    num_proc=os.cpu_count()
)

print("train_long 업로드 중...")
dataset_dict['train_long'].push_to_hub(
    "brainer/non_contractual_legal_data", 
    config_name="long",
    split="train",
    num_proc=os.cpu_count()
)

print("valid_long 업로드 중...")
dataset_dict['valid_long'].push_to_hub(
    "brainer/non_contractual_legal_data", 
    config_name="long",
    split="validation",
    num_proc=os.cpu_count()
)

print("✓ 업로드 완료!")

train_short 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 61.23ba/s]
Processing Files (1 / 1): 100%|██████████| 1.18MB / 1.18MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 52.95ba/s]
Processing Files (1 / 1): 100%|██████████| 2.10MB / 2.10MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s    | 2/16 [00:02<00:14,  1.05s/ shards]
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 31.49ba/s]
Processing Files (1 / 1): 100%|██████████| 2.67MB / 2.67MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Processing Files (1 / 1): 100%|██████████|  854kB /  854kB,  0.00B/s  02<00:08,  1.54 shards/s]
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Processing Files (1 / 1): 100%|██████████| 2.34MB / 2.34MB,  0.00B/s  02<00:05,  2.01 shards/s]
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Processing Files (1 /

valid_short 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 145.09ba/s]
Processing Files (1 / 1): 100%|██████████|  362kB /  362kB,  201kB/s  
New Data Upload: 100%|██████████|  193kB /  193kB,  107kB/s  
Processing Files (1 / 1): 100%|██████████|  117kB /  117kB, 58.4kB/s  03<00:50,  3.39s/ shards]
New Data Upload: 100%|██████████|  117kB /  117kB, 58.4kB/s  
Processing Files (1 / 1): 100%|██████████|  261kB /  261kB,  131kB/s  03<00:20,  1.48s/ shards]
New Data Upload: 100%|██████████|  130kB /  130kB, 65.2kB/s  
Processing Files (1 / 1): 100%|██████████|  229kB /  229kB,  114kB/s  
New Data Upload: 100%|██████████|  229kB /  229kB,  114kB/s  
Processing Files (1 / 1): 100%|██████████|  319kB /  319kB,  159kB/s  03<00:07,  1.64 shards/s]
Processing Files (1 / 1): 100%|██████████|  346kB /  346kB,  157kB/s  
New Data Upload: 100%|██████████|  198kB /  198kB, 99.0kB/s  
New Data Upload: 100%|██████████|  257kB /  257kB,  117kB/s  
Processing Files (1 / 1): 100%|██████████|  

train_long 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 40.58ba/s]
Processing Files (1 / 1): 100%|██████████| 1.25MB / 1.25MB,  522kB/s  
New Data Upload: 100%|██████████|  460kB /  460kB,  192kB/s  
Processing Files (1 / 1): 100%|██████████|  890kB /  890kB,  404kB/s  03<00:56,  3.77s/ shards]
New Data Upload: 100%|██████████|  374kB /  374kB,  170kB/s  
Processing Files (1 / 1): 100%|██████████| 1.98MB / 1.98MB,  826kB/s  
New Data Upload: 100%|██████████|  790kB /  790kB,  329kB/s  
Processing Files (1 / 1): 100%|██████████| 2.59MB / 2.59MB, 1.08MB/s  04<00:14,  1.09s/ shards]
New Data Upload: 100%|██████████|  944kB /  944kB,  394kB/s  
Processing Files (1 / 1): 100%|██████████| 1.90MB / 1.90MB,  790kB/s  04<00:09,  1.27 shards/s]
New Data Upload: 100%|██████████|  780kB /  780kB,  325kB/s  
Processing Files (1 / 1): 100%|██████████| 2.37MB / 2.37MB,  910kB/s  04<00:06,  1.59 shards/s]
New Data Upload: 100%|██████████|  921kB /  921kB,  354kB/s  
Processing Files (1 

valid_long 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 120.90ba/s]
Processing Files (1 / 1): 100%|██████████|  229kB /  229kB,  127kB/s  
New Data Upload: 100%|██████████|  193kB /  193kB,  107kB/s  
Processing Files (1 / 1): 100%|██████████|  264kB /  264kB,  132kB/s  03<00:48,  3.24s/ shards]
New Data Upload: 100%|██████████|  254kB /  254kB,  127kB/s  
Processing Files (1 / 1): 100%|██████████|  165kB /  165kB, 82.6kB/s  03<00:21,  1.54s/ shards]
New Data Upload: 100%|██████████|  143kB /  143kB, 71.4kB/s  
Processing Files (1 / 1): 100%|██████████|  234kB /  234kB,  117kB/s  
New Data Upload: 100%|██████████|  234kB /  234kB,  117kB/s  
Processing Files (1 / 1): 100%|██████████|  343kB /  343kB,  156kB/s  
New Data Upload: 100%|██████████|  317kB /  317kB,  144kB/s  
Processing Files (1 / 1): 100%|██████████|  302kB /  302kB,  151kB/s  03<00:05,  2.13 shards/s]
New Data Upload: 100%|██████████|  185kB /  185kB, 92.5kB/s  
Processing Files (1 / 1): 100%|██████████|  

✓ 업로드 완료!
